## CryptoBot - data understanding streaming

#### 1. Import Libraries & API Keys

In [11]:
#from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
import websocket
import pandas as pd
import json
import os
import threading
import os

In [12]:
load_dotenv()
binance_api_key = os.getenv("BINANCE_API_KEY")
binance_api_secret = os.getenv("BINANCE_API_SECRET")
client = Client(binance_api_key, binance_api_secret, testnet=True)

## Streaming Data (Websocket Market Endpoint)
###### https://binance-docs.github.io/apidocs/spot/en/#websocket-market-streams

### Trade based

These data reflect the completed transactions where buyers and sellers have executed their orders on the platform.

In [17]:
df = pd.DataFrame()

def on_message(ws, message):
    msg = json.loads(message)
    d = [(msg['T'],msg['p'])]
    global df
    df = pd.concat([df, pd.DataFrame.from_records(d)])
    #print(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print('### closed ###')
    df.columns = ['time', 'price']
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    print_to_json_file(filename,df)
    
def on_open(ws):
    print("Opened connection")
    # Start a timer to close the WebSocket after 40 seconds
    def stop_stream():
        print("Closing WebSocket after 40 seconds...")
        ws.close()
    
    timer = threading.Timer(5, stop_stream)
    timer.start()

def print_to_json_file(filename,df):
    
    json_str = df.to_json(orient='records', date_format="iso")  

    with open(filename, "w") as file:
        file.write(json_str)
    print("Text file written!", filename)

### run script trades

In [18]:
symbol_array = ['btcusdt','ethbtc']
path = "data/streaming_data/trades"
os.makedirs(path, exist_ok=True)

for symbol in symbol_array:
    socket = f'wss://stream.binance.com:9443/ws/{symbol}@trade'
    filename = path + '/' + symbol.upper() +'_hist_trades.json'
    ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_error=on_error,on_close=on_close)
    ws.run_forever()

Opened connection
Closing WebSocket after 40 seconds...
### closed ###
Text file written! data/streaming_data/trades/BTCUSDT_hist_trades.json
Opened connection
Closing WebSocket after 40 seconds...
### closed ###
Length mismatch: Expected axis has 4 elements, new values have 2 elements
